In [1]:
!wget https://raw.githubusercontent.com/EeYeoKeat/Movie-Recommender-Systems-With-Keras/master/ml-latest-small/movies.csv
!wget https://raw.githubusercontent.com/EeYeoKeat/Movie-Recommender-Systems-With-Keras/master/ml-latest-small/ratings.csv

--2019-09-19 11:53:43--  https://raw.githubusercontent.com/EeYeoKeat/Movie-Recommender-Systems-With-Keras/master/ml-latest-small/movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 494431 (483K) [text/plain]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 482.84K  --.-KB/s    in 0.02s   

2019-09-19 11:53:43 (25.3 MB/s) - ‘movies.csv’ saved [494431/494431]

--2019-09-19 11:53:46--  https://raw.githubusercontent.com/EeYeoKeat/Movie-Recommender-Systems-With-Keras/master/ml-latest-small/ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, aw

In [2]:
!ls

movies.csv  ratings.csv  sample_data


In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
ratings = pd.read_csv("ratings.csv")

In [12]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
movies = pd.read_csv("movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
g = ratings.groupby('userId')['rating'].count()
top_users = g.sort_values(ascending=False)[:15]

g = ratings.groupby('movieId')['rating'].count()
top_movies = g.sort_values(ascending=False)[:15]

top_r = ratings.join(top_users, rsuffix='_r', how='inner', on='userId')
top_r = top_r.join(top_movies, rsuffix='_r', how='inner', on='movieId')

pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,1,50,110,260,296,318,356,480,527,589,593,1196,2571,2858,2959
userId,,,,,,,,,,,,,,,
68,2.5,3.0,2.5,5.0,2.0,3.0,3.5,3.5,4.0,3.5,3.5,5.0,4.5,5.0,2.5
182,4.0,4.5,3.5,3.5,5.0,4.5,5.0,3.5,4.0,2.0,4.5,3.0,5.0,5.0,5.0
249,4.0,4.0,5.0,5.0,4.0,4.5,4.5,4.0,4.5,4.0,4.0,5.0,5.0,4.5,5.0
274,4.0,4.0,4.5,3.0,5.0,4.5,4.5,3.5,4.0,4.5,4.0,4.5,4.0,5.0,5.0
288,4.5,NaN,5.0,5.0,5.0,5.0,5.0,2.0,5.0,4.0,5.0,4.5,3.0,NaN,3.5
307,4.0,4.5,3.5,3.5,4.5,4.5,4.0,3.5,4.5,2.5,4.5,3.0,3.5,4.0,4.0
380,5.0,4.0,4.0,5.0,5.0,3.0,5.0,5.0,NaN,5.0,5.0,5.0,4.5,NaN,4.0
387,NaN,4.5,3.5,4.5,5.0,3.5,4.0,3.0,NaN,3.5,4.0,4.5,4.0,4.5,4.5
414,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0


In [15]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

item_enc = LabelEncoder()
ratings['movie'] = item_enc.fit_transform(ratings['movieId'].values)
n_movies = ratings['movie'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

n_users, n_movies, min_rating, max_rating

(610, 9724, 0.5, 5.0)

In [0]:
from sklearn.model_selection import train_test_split

In [19]:
X = ratings[['user', 'movie']].values
y = ratings['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((75627, 2), (25209, 2), (75627,), (25209,))

In [0]:
n_factors = 50

X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(movie)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])

    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    return model

In [26]:
model = RecommenderV1(n_users, n_movies, n_factors)
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 50)        30500       input_2[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 50)        486200      input_3[0][0]                    
__________________________________________________________________________________________

In [27]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=10,
                    verbose=1, validation_data=(X_test_array, y_test))



Train on 75627 samples, validate on 25209 samples
Epoch 1/10
75627/75627 [==============================] - 10s 128us/step - loss: 10.3495 - val_loss: 4.1470
Epoch 2/10
75627/75627 [==============================] - 4s 51us/step - loss: 2.4100 - val_loss: 1.8805
Epoch 3/10
75627/75627 [==============================] - 4s 52us/step - loss: 1.2581 - val_loss: 1.4842
Epoch 4/10
75627/75627 [==============================] - 4s 51us/step - loss: 0.9194 - val_loss: 1.3600
Epoch 5/10
75627/75627 [==============================] - 4s 51us/step - loss: 0.7575 - val_loss: 1.3109
Epoch 6/10
75627/75627 [==============================] - 4s 52us/step - loss: 0.6612 - val_loss: 1.2937
Epoch 7/10
75627/75627 [==============================] - 4s 52us/step - loss: 0.5919 - val_loss: 1.2910
Epoch 8/10
75627/75627 [==============================] - 4s 52us/step - loss: 0.5357 - val_loss: 1.2944
Epoch 9/10
75627/75627 [==============================] - 4s 54us/step - loss: 0.4821 - val_loss: 1.2991


In [0]:
from keras.layers import Add, Activation, Lambda

class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x

def RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    mb = EmbeddingLayer(n_movies, 1)(movie)

    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)

    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    return model

In [29]:
model = RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 50)        30500       input_4[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        486200      input_5[0][0]                    
____________________________________________________________________________________________

In [31]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=10,
                    verbose=1, validation_data=(X_test_array, y_test))

Train on 75627 samples, validate on 25209 samples
Epoch 1/10
75627/75627 [==============================] - 5s 64us/step - loss: 0.2388 - val_loss: 0.7766
Epoch 2/10
75627/75627 [==============================] - 5s 63us/step - loss: 0.1904 - val_loss: 0.7952
Epoch 3/10
75627/75627 [==============================] - 5s 64us/step - loss: 0.1556 - val_loss: 0.8164
Epoch 4/10
75627/75627 [==============================] - 5s 62us/step - loss: 0.1295 - val_loss: 0.8342
Epoch 5/10
75627/75627 [==============================] - 5s 64us/step - loss: 0.1101 - val_loss: 0.8532
Epoch 6/10
75627/75627 [==============================] - 5s 64us/step - loss: 0.0955 - val_loss: 0.8702
Epoch 7/10
75627/75627 [==============================] - 5s 64us/step - loss: 0.0842 - val_loss: 0.8857
Epoch 8/10
75627/75627 [==============================] - 5s 65us/step - loss: 0.0756 - val_loss: 0.9009
Epoch 9/10
75627/75627 [==============================] - 5s 63us/step - loss: 0.0684 - val_loss: 0.9152
Epoch

In [0]:
from keras.layers import Concatenate, Dense, Dropout

def RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    
    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)

    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    return model

In [33]:
model = RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 50)        30500       input_6[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 50)        486200      input_7[0][

In [34]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=10,
                    verbose=1, validation_data=(X_test_array, y_test))

Train on 75627 samples, validate on 25209 samples
Epoch 1/10
75627/75627 [==============================] - 6s 77us/step - loss: 0.9496 - val_loss: 0.7823
Epoch 2/10
75627/75627 [==============================] - 5s 68us/step - loss: 0.7744 - val_loss: 0.7629
Epoch 3/10
75627/75627 [==============================] - 5s 69us/step - loss: 0.7314 - val_loss: 0.7622
Epoch 4/10
75627/75627 [==============================] - 5s 69us/step - loss: 0.7075 - val_loss: 0.7670
Epoch 5/10
75627/75627 [==============================] - 5s 69us/step - loss: 0.6935 - val_loss: 0.7708
Epoch 6/10
75627/75627 [==============================] - 5s 68us/step - loss: 0.6807 - val_loss: 0.7742
Epoch 7/10
75627/75627 [==============================] - 5s 69us/step - loss: 0.6806 - val_loss: 0.7750
Epoch 8/10
75627/75627 [==============================] - 5s 69us/step - loss: 0.6721 - val_loss: 0.7795
Epoch 9/10
75627/75627 [==============================] - 5s 70us/step - loss: 0.6673 - val_loss: 0.7781
Epoch